# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Training Data & Feature views</span>

<span style="font-width:bold; font-size: 1.4rem;">This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## 🗒️ In this notebook we will see how to create a training dataset from the feature groups: 

1. Retrieving Feature Groups
2. Feature Group investigation
3. Transformation functions
4. Feature Views
5. Training Datasets
6. Training Datasets with Event Time filter



![tutorial-flow](images/02_training-dataset.png) 

---

## <span style="color:#ff5f27;"> 🔮 🪝 Connecting to Feature Store and Retrieving Feature Groups </span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://ed408010-19ac-11ed-9333-c9e9623aac8a.cloud.hopsworks.ai/p/120
Connected. Call `.close()` to terminate connection gracefully.


> In order to retrieve necessary Feature Group we can use `FeatureStore.get_or_create_feature_group()` method.

In [2]:
fg_weather = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

In [3]:
fg_calendar = fs.get_or_create_feature_group(
    name = 'calendar_fg',
    version = 1
)

In [4]:
fg_electricity = fs.get_or_create_feature_group(
    name = 'electricity_fg',
    version = 1
)

---

# <span style="color:#ff5f27;">🕵🏻‍♂️ Feature Groups Investigation</span>

We can use `FeatureGroup.show()` method to select top n rows. 

Also we use method `FeatureGroup.read()` in order **to aggregate queries**, which are the output of next methods:

- `FeatureGroup.get_feature()` to get specific feature from our Feature Group.

- `FeatureGroup.select()` to get a subset of features from our Feature Group.

- `FeatureGroup.select_all()` to get all features from our Feature Group.

- `FeatureGroup.select_except()` to get all features except a few from our Feature Group.

- `FeatureGroup.filter()` to apply specific filter to the feature group.

In [5]:
fg_weather.select_all()

In [6]:
fg_weather.select_all().read().head()

2022-08-21 21:55:09,666 INFO: USE `electricity_featurestore`
2022-08-21 21:55:11,304 INFO: SELECT `fg0`.`index` `index`, `fg0`.`date` `date`, `fg0`.`min_temperature` `min_temperature`, `fg0`.`max_temperature` `max_temperature`, `fg0`.`solar_exposure` `solar_exposure`, `fg0`.`rainfall` `rainfall`, `fg0`.`day_of_week` `day_of_week`, `fg0`.`day_of_month` `day_of_month`, `fg0`.`day_of_year` `day_of_year`, `fg0`.`week_of_year` `week_of_year`, `fg0`.`month` `month`, `fg0`.`quarter` `quarter`, `fg0`.`year` `year`
FROM `electricity_featurestore`.`weather_fg_1` `fg0`


,index,date,min_temperature,max_temperature,solar_exposure,rainfall,day_of_week,day_of_month,day_of_year,week_of_year,month,quarter,year
0,1886,1583020800000,13.2,32.1,20.8,0.0,6,1,61,9,3,1,2020
1,2039,1596240000000,5.7,16.4,11.1,0.0,5,1,214,31,8,3,2020
2,1347,1536451200000,8.6,18.1,7.3,0.0,6,9,252,36,9,3,2018
3,634,1474848000000,8.1,16.9,13.6,0.6,0,26,270,39,9,3,2016
4,8,1420761600000,16.5,18.0,3.1,1.2,4,9,9,2,1,1,2015


In [7]:
fg_calendar.select_except(['index']).show(5)

2022-08-21 21:55:16,165 INFO: USE `electricity_featurestore`
2022-08-21 21:55:16,865 INFO: SELECT `fg0`.`date` `date`, `fg0`.`school_day` `school_day`, `fg0`.`holiday` `holiday`
FROM `electricity_featurestore`.`calendar_fg_1` `fg0`


,date,school_day,holiday
0,1583020800000,1,0
1,1596240000000,1,0
2,1536451200000,1,0
3,1474848000000,0,0
4,1420761600000,0,0


In [8]:
fg_electricity.select('demand').show(5)

2022-08-21 21:55:18,618 INFO: USE `electricity_featurestore`
2022-08-21 21:55:19,310 INFO: SELECT `fg0`.`demand` `demand`
FROM `electricity_featurestore`.`electricity_fg_1` `fg0`


,demand
0,101413.145
1,111590.650
2,107838.950
3,124343.190
4,135452.260


In [9]:
fg_electricity.filter(fg_electricity.demand > 10000).show(5)

2022-08-21 21:55:21,145 INFO: USE `electricity_featurestore`
2022-08-21 21:55:21,852 INFO: SELECT `fg0`.`index` `index`, `fg0`.`date` `date`, `fg0`.`rrp` `rrp`, `fg0`.`frac_at_neg_rrp` `frac_at_neg_rrp`, `fg0`.`demand` `demand`, `fg0`.`rrp_positive` `rrp_positive`, `fg0`.`demand_neg_rrp` `demand_neg_rrp`, `fg0`.`rrp_negative` `rrp_negative`, `fg0`.`demand_pos_rrp` `demand_pos_rrp`, `fg0`.`demand_7_mean` `demand_7_mean`, `fg0`.`demand_7_std` `demand_7_std`, `fg0`.`demand_14_mean` `demand_14_mean`, `fg0`.`demand_14_std` `demand_14_std`, `fg0`.`demand_30_mean` `demand_30_mean`, `fg0`.`demand_30_std` `demand_30_std`
FROM `electricity_featurestore`.`electricity_fg_1` `fg0`
WHERE `fg0`.`demand` > 10000


,index,date,rrp,frac_at_neg_rrp,demand,rrp_positive,demand_neg_rrp,rrp_negative,demand_pos_rrp,demand_7_mean,demand_7_std,demand_14_mean,demand_14_std,demand_30_mean,demand_30_std
0,1886,1583020800000,43.745921,0.000000,101413.145,43.745921,0.000,0.000000,101413.145,110717.865714,9220.740759,110795.223214,8592.274480,113918.470167,11661.928533
1,2039,1596240000000,47.915895,0.000000,111590.650,47.915895,0.000,0.000000,111590.650,127876.293571,7974.452809,130397.305714,8870.056804,131883.940500,8381.503670
2,1347,1536451200000,67.615591,0.104167,107838.950,77.332217,9927.775,-28.213155,97911.175,119398.098571,8876.580027,124830.694643,10492.613383,125011.080833,9375.466267
3,634,1474848000000,47.671470,0.000000,124343.190,47.671470,0.000,0.000000,124343.190,120814.523571,10247.936071,122961.179286,10768.576673,123261.361000,9562.472546
4,1435,1544054400000,125.308108,0.000000,135452.260,125.308108,0.000,0.000000,135452.260,109817.412143,14171.194821,108530.186071,10633.545758,107938.950000,9656.161129


---

# <span style="color:#ff5f27;">🧑🏻‍🔬 Transformation functions</span>

Hopsworks Feature Store provides functionality to attach transformation functions to training datasets.

Hopsworks Feature Store also comes with built-in transformation functions such as `min_max_scaler`, `standard_scaler`, `robust_scaler` and `label_encoder`.

In [10]:
[t_func.name for t_func in fs.get_transformation_functions()]

['min_max_scaler', 'standard_scaler', 'robust_scaler', 'label_encoder']

We can retrieve transformation function we need .

To attach transformation function to training dataset provide transformation functions as dict, where key is feature name and value is online transformation function name.

Also training dataset must be created from the Query object. Once attached transformation function will be applied on whenever save, insert and get_serving_vector methods are called on training dataset object.

In [11]:
# Load transformation functions.
standard_scaler = fs.get_transformation_function(name = 'standard_scaler')
label_encoder = fs.get_transformation_function(name = 'label_encoder')

#Map features to transformations.
mapping_transformers = {
    "rrp_positive": standard_scaler,
    "rrp_negative": standard_scaler,
    "school_day": label_encoder,
    "holiday": label_encoder
}

---

## <span style="color:#ff5f27;">💼 Query Preparation</span>

In [12]:
fg_weather.select_all().join(
                            fg_calendar.select_all()).show(5)

2022-08-21 21:55:26,308 INFO: USE `electricity_featurestore`
2022-08-21 21:55:27,043 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`index` `index`, `fg1`.`date` `date`, `fg1`.`min_temperature` `min_temperature`, `fg1`.`max_temperature` `max_temperature`, `fg1`.`solar_exposure` `solar_exposure`, `fg1`.`rainfall` `rainfall`, `fg1`.`day_of_week` `day_of_week`, `fg1`.`day_of_month` `day_of_month`, `fg1`.`day_of_year` `day_of_year`, `fg1`.`week_of_year` `week_of_year`, `fg1`.`month` `month`, `fg1`.`quarter` `quarter`, `fg1`.`year` `year`, `fg1`.`index` `join_pk_index`, `fg1`.`date` `join_evt_date`, `fg0`.`school_day` `school_day`, `fg0`.`holiday` `holiday`, RANK() OVER (PARTITION BY `fg0`.`index`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `electricity_featurestore`.`weather_fg_1` `fg1`
INNER JOIN `electricity_featurestore`.`calendar_fg_1` `fg0` ON `fg1`.`index` = `fg0`.`index` AND `fg1`.`date` >= `fg0`.`date`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_f

,index,date,min_temperature,max_temperature,solar_exposure,rainfall,day_of_week,day_of_month,day_of_year,week_of_year,month,quarter,year,school_day,holiday
0,0,1420070400000,13.3,26.9,23.6,0.0,3,1,1,1,1,1,2015,0,1
1,0,1602115200000,24.3,26.0,1.8,6.6,3,8,282,41,10,4,2020,1,0
2,1,1420156800000,15.4,38.8,26.8,0.0,4,2,2,1,1,1,2015,0,0
3,1,1602201600000,21.1,33.0,15.1,-4.8,4,9,283,41,10,4,2020,0,1
4,2,1420243200000,20.0,38.2,26.5,0.0,5,3,3,1,1,1,2015,0,0


In [13]:
fg_query = fg_weather.select_all()\
                        .join(
                            fg_calendar.select_all(),
                            on = ['index']
                        )\
                        .join(
                            fg_electricity.select_all(),
                            on = ['index']
                        )
fg_query.show(5)

2022-08-21 21:55:51,541 INFO: USE `electricity_featurestore`
2022-08-21 21:55:52,238 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg2`.`index` `index`, `fg2`.`date` `date`, `fg2`.`min_temperature` `min_temperature`, `fg2`.`max_temperature` `max_temperature`, `fg2`.`solar_exposure` `solar_exposure`, `fg2`.`rainfall` `rainfall`, `fg2`.`day_of_week` `day_of_week`, `fg2`.`day_of_month` `day_of_month`, `fg2`.`day_of_year` `day_of_year`, `fg2`.`week_of_year` `week_of_year`, `fg2`.`month` `month`, `fg2`.`quarter` `quarter`, `fg2`.`year` `year`, `fg2`.`index` `join_pk_index`, `fg2`.`date` `join_evt_date`, `fg0`.`school_day` `school_day`, `fg0`.`holiday` `holiday`, RANK() OVER (PARTITION BY `fg2`.`index`, `fg2`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `electricity_featurestore`.`weather_fg_1` `fg2`
INNER JOIN `electricity_featurestore`.`calendar_fg_1` `fg0` ON `fg2`.`index` = `fg0`.`index` AND `fg2`.`date` >= `fg0`.`date`) NA
WHERE `pit_rank_hopsworks` = 1), right_fg1 AS (S

,index,date,min_temperature,max_temperature,solar_exposure,rainfall,day_of_week,day_of_month,day_of_year,week_of_year,...,rrp_positive,demand_neg_rrp,rrp_negative,demand_pos_rrp,demand_7_mean,demand_7_std,demand_14_mean,demand_14_std,demand_30_mean,demand_30_std
0,33,1604966400000,17.4,37.1,34.3,-4.8,1,10,315,46,...,83.200000,6956.5,35.1,90990.400,125004.357143,23951.277370,123192.721429,23761.598927,117832.410000,21241.786982
1,34,1423008000000,14.8,19.0,21.2,0.0,2,4,35,6,...,27.809134,0.0,0.0,117398.030,114758.001429,8948.957583,117702.133929,14693.328802,120100.977833,15883.964829
2,35,1423094400000,13.3,24.2,28.8,0.0,3,5,36,6,...,31.384571,0.0,0.0,122279.425,115181.672143,9265.201226,115491.228571,10730.117628,119821.209167,15764.813039
3,36,1423180800000,14.5,35.7,28.4,0.0,4,6,37,6,...,38.840180,0.0,0.0,141837.005,118155.794286,13720.958427,115758.500357,11364.319495,119431.948667,15031.278558
4,36,1605225600000,8.5,12.1,-5.4,14.1,4,13,318,46,...,-186.200000,1627.5,-49.7,147921.500,122091.871429,23885.930012,119959.142857,23762.544203,117822.043333,22225.094507


---

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

`Feature Views` stands between **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create **Feature Views** which store a metadata of our data. Having **Feature Views** we can create **Training Dataset**.

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.

In order to create Feature View we can use `FeatureStore.create_feature_view()` method.

We can specify next parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- our target variable.

- `transformation_functions` - functions to transform our features.

- `query` - query object with data.

In [14]:
feature_view = fs.create_feature_view(
    name = 'electricity_feature_view',
    version = 1,
    labels = ['demand'],
    query = fg_query
)

Feature view created successfully, explore it at 
https://ed408010-19ac-11ed-9333-c9e9623aac8a.cloud.hopsworks.ai/p/120/fs/68/fv/electricity_feature_view/version/1


In [15]:
feature_view

For now `Feature View` is saved in Hopsworks and we can retrieve it using `FeatureStore.get_feature_view()`.

In [16]:
feature_view = fs.get_feature_view(
    name = 'electricity_feature_view',
    version = 1
)

In [17]:
feature_view.version

1

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

To create training dataset we use `FeatureView.create_training_data()` method.

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- We can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

#### <span style="color:#ff5f27;"> ⛳️ Simple Training Dataset</span>

In [18]:
feature_view.create_training_data(
    description = 'training_dataset',
    data_format = 'csv'
)

Training dataset job started successfully, you can follow the progress at 
https://ed408010-19ac-11ed-9333-c9e9623aac8a.cloud.hopsworks.ai/p/120/jobs/named/electricity_feature_view_1_1_create_fv_td_21082022215620/executions


(1, <hsfs.core.job.Job at 0x7f17bc8ff070>)

- We can create **train, test** splits using `create_train_test_split()`. 

- We can create **train,validation, test** splits using `create_train_validation_test_splits()` methods.

- The only thing is that we should specify desired ratio of splits.

#### <span style="color:#ff5f27;"> ⛳️ Dataset with train and test splits</span>

In [19]:
feature_view.create_train_test_split(
    test_size = 0.2
)

Training dataset job started successfully, you can follow the progress at 
https://ed408010-19ac-11ed-9333-c9e9623aac8a.cloud.hopsworks.ai/p/120/jobs/named/electricity_feature_view_1_2_create_fv_td_21082022215904/executions


(2, <hsfs.core.job.Job at 0x7f17bc8ff160>)

#### <span style="color:#ff5f27;"> ⛳️ Dataset with train, validation and test splits</span>

In [20]:
feature_view.create_train_validation_test_split(
    validation_size = 0.2,
    test_size = 0.1
)

Training dataset job started successfully, you can follow the progress at 
https://ed408010-19ac-11ed-9333-c9e9623aac8a.cloud.hopsworks.ai/p/120/jobs/named/electricity_feature_view_1_3_create_fv_td_21082022220324/executions


(3, <hsfs.core.job.Job at 0x7f17bc8ff850>)

---

## <span style="color:#ff5f27;"> 🪝 Retrieving Datasets </span>

#### <span style="color:#ff5f27;"> ⛳️ Simple Training Dataset</span>

In [21]:
X_train, y_train = feature_view.get_training_data(
    training_dataset_version = 1
)

In [22]:
X_train.head()

,index,date,min_temperature,max_temperature,solar_exposure,rainfall,day_of_week,day_of_month,day_of_year,week_of_year,...,rrp_positive,demand_neg_rrp,rrp_negative,demand_pos_rrp,demand_7_mean,demand_7_std,demand_14_mean,demand_14_std,demand_30_mean,demand_30_std
0,154,1433376000000,4.4,11.3,5.5,0.0,3,4,155,23,...,33.050285,0.000,0.0,147155.265,134826.100714,15277.541585,132421.544286,12121.312927,129020.392667,11456.502219
1,573,1469577600000,7.5,12.9,6.9,0.4,2,27,209,30,...,36.397463,0.000,0.0,143209.755,133247.215000,10620.203460,132380.252857,10598.615006,132709.017833,10018.226341
2,635,1474934400000,7.4,14.8,13.4,12.6,1,27,271,39,...,49.794504,0.000,0.0,126888.770,121418.018571,10496.400154,122401.794643,10304.147294,123663.239167,9448.684467
3,654,1476576000000,17.4,23.1,8.0,0.0,6,16,290,41,...,22.474960,1740.595,-3.6,100052.440,117109.188571,12509.209925,115471.599286,11851.022854,116607.459500,10873.536035
4,872,1495411200000,10.4,20.3,10.1,0.0,0,22,142,21,...,72.931488,0.000,0.0,113551.040,118247.162143,8599.465978,119668.648214,8471.561767,116976.748167,9751.151336


In [23]:
y_train.head()

,demand
0,147155.265
1,143209.755
2,126888.770
3,101793.035
4,113551.040


In [24]:
X_train.shape

(2094, 27)

#### <span style="color:#ff5f27;"> ⛳️ Dataset with train and test splits</span>

In [25]:
X_train, y_train, X_test, y_test = feature_view.get_train_test_split(
    training_dataset_version = 2
)

In [26]:
X_train.head()

,index,date,min_temperature,max_temperature,solar_exposure,rainfall,day_of_week,day_of_month,day_of_year,week_of_year,...,rrp_positive,demand_neg_rrp,rrp_negative,demand_pos_rrp,demand_7_mean,demand_7_std,demand_14_mean,demand_14_std,demand_30_mean,demand_30_std
0,154,1433376000000,4.4,11.3,5.5,0.0,3,4,155,23,...,33.050285,0.000,0.0,147155.265,134826.100714,15277.541585,132421.544286,12121.312927,129020.392667,11456.502219
1,573,1469577600000,7.5,12.9,6.9,0.4,2,27,209,30,...,36.397463,0.000,0.0,143209.755,133247.215000,10620.203460,132380.252857,10598.615006,132709.017833,10018.226341
2,635,1474934400000,7.4,14.8,13.4,12.6,1,27,271,39,...,49.794504,0.000,0.0,126888.770,121418.018571,10496.400154,122401.794643,10304.147294,123663.239167,9448.684467
3,654,1476576000000,17.4,23.1,8.0,0.0,6,16,290,41,...,22.474960,1740.595,-3.6,100052.440,117109.188571,12509.209925,115471.599286,11851.022854,116607.459500,10873.536035
4,872,1495411200000,10.4,20.3,10.1,0.0,0,22,142,21,...,72.931488,0.000,0.0,113551.040,118247.162143,8599.465978,119668.648214,8471.561767,116976.748167,9751.151336


In [27]:
X_train.shape

(1655, 27)

In [28]:
X_test.shape

(439, 27)

#### <span style="color:#ff5f27;"> ⛳️ Dataset with train, validation and test splits</span>

In [29]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_view.get_train_validation_test_split(
    training_dataset_version = 3
)

In [30]:
X_train.head()

,index,date,min_temperature,max_temperature,solar_exposure,rainfall,day_of_week,day_of_month,day_of_year,week_of_year,...,rrp_positive,demand_neg_rrp,rrp_negative,demand_pos_rrp,demand_7_mean,demand_7_std,demand_14_mean,demand_14_std,demand_30_mean,demand_30_std
0,154,1433376000000,4.4,11.3,5.5,0.0,3,4,155,23,...,33.050285,0.0,0.0,147155.265,134826.100714,15277.541585,132421.544286,12121.312927,129020.392667,11456.502219
1,573,1469577600000,7.5,12.9,6.9,0.4,2,27,209,30,...,36.397463,0.0,0.0,143209.755,133247.215000,10620.203460,132380.252857,10598.615006,132709.017833,10018.226341
2,635,1474934400000,7.4,14.8,13.4,12.6,1,27,271,39,...,49.794504,0.0,0.0,126888.770,121418.018571,10496.400154,122401.794643,10304.147294,123663.239167,9448.684467
3,971,1503964800000,5.4,12.9,9.1,0.0,1,29,241,35,...,143.848040,0.0,0.0,139814.040,130599.129286,10117.240137,129806.963571,8575.738851,129232.795833,10549.863870
4,1843,1579305600000,13.9,29.4,27.2,0.0,5,18,18,3,...,52.053155,0.0,0.0,105946.340,115698.075714,16901.740872,112640.898571,14770.620802,110161.665000,15941.806878


---

## <span style="color:#ff5f27;"> 🔮 Creating Training Datasets with Event Time filter</span>

First of all lets import **datetime** from datetime library and set up a time format.

Then we can define start_time point and end_time point.

Finally we can create training dataset with data in specific time bourders. 


In [31]:
from datetime import datetime

def from_unix_to_datetime(unix):
    return datetime.utcfromtimestamp(unix).strftime('%Y-%m-%d %H:%M:%S')

In [32]:
date_format = '%Y-%m-%d %H:%M:%S'

start_time_train = int(float(datetime.strptime('2017-01-01 00:00:01',date_format).timestamp()) * 1000)
end_time_train = int(float(datetime.strptime('2018-02-01 23:59:59',date_format).timestamp()) * 1000)

start_time_test = int(float(datetime.strptime('2018-02-02 23:59:59',date_format).timestamp()) * 1000)
end_time_test = int(float(datetime.strptime('2019-02-01 23:59:59',date_format).timestamp()) * 1000)

#### <span style="color:#ff5f27;"> ⛳️ Simple Training Dataset with event time</span>

In [33]:
feature_view.create_training_data(
    description = 'data_2017_2018',
    data_format = 'csv',
    start_time = start_time_train,
    end_time = end_time_train
)

Training dataset job started successfully, you can follow the progress at 
https://ed408010-19ac-11ed-9333-c9e9623aac8a.cloud.hopsworks.ai/p/120/jobs/named/electricity_feature_view_1_4_create_fv_td_21082022221121/executions


(4, <hsfs.core.job.Job at 0x7f1772ce7fd0>)

In [34]:
X_train_lim, y_train_lim = feature_view.get_training_data(
    training_dataset_version = 4
)

#### <span style="color:#ff5f27;"> ⛳️ Training Dataset with train and test splits with event time</span>

In [35]:
# feature_view.create_train_test_split(
#     test_size = 0.2,
#     train_start = start_time_train,
#     train_end = end_time_train,
#     test_start = start_time_test,
#     test_end = end_time_test
# )

---

### <span style="color:#ff5f27;"> Next Steps</span>

In the next notebook, we will train a model on the Training Dataset we created in this notebook.